In [1]:
import weave
weave.capture_weave_server_logs()

# weave.use_fixed_server_port()

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import random

@weave.op(
    name="root-sin_image",
    render_info={"type": "function"},
    input_type={"f": weave.types.Int()},
    output_type=weave.types.FileType(weave.types.Const(weave.types.String(), "png")),
)
def sin_image(f):    
    x = np.arange(0, f * np.pi, 0.1)
    y = np.sin(x)
    path = "/tmp/my-%s.png" % random.randrange(0, 1000000)
    plt.rcParams["figure.figsize"] = (22, 7)
    plt.plot(x, y)
    plt.savefig(path)
    plt.close()
    return weave.ops.LocalFile(path)

In [4]:
x = weave.save(7, name='my-number')

im = sin_image(x + 2)
weave.show(im)

In [ ]:
import json
class RunType(weave.types.BasicType):
    name = 'run'

@weave.op(
    name="root-run_accuracy_barchart",
    render_info={"type": "function"},
    input_type={"runs": weave.types.List(weave.ops.RunType())},
    output_type=weave.types.FileType(weave.types.Const(weave.types.String(), 'png'))
)
def run_accuracy_barchart(runs):
    names = [r['name'] for r in runs]
    xs = range(len(runs))
    ys = [r['summary'].get('acc') for r in runs]
    
    plt.rcParams["figure.figsize"] = (22, 5)
    plt.bar(xs, ys, align='center', alpha=0.5)
    plt.xticks(xs, names)
    plt.ylabel('acc')
    plt.title('Run accuracies')
    path = "/tmp/my-%s.png" % random.randrange(0, 1000000)
    plt.savefig(path)
    plt.close()
    return weave.ops.LocalFile(path) 

In [ ]:
runs = weave.ops.project('shawn', 'fasion-sweep').runs()
run_accuracy_barchart(runs)